## Kenneth Lay's Network to CSV

In [1]:
import os
import pandas as pd

In [2]:
kmetadata = pd.DataFrame()
kexchange = pd.DataFrame()
kbodies = pd.DataFrame()

In [3]:
# starting point
emails_path = 'C:/Users/Aneesha2/Desktop/enron_contents/maildir/lay-k/sent/'
    
for email in os.listdir(emails_path):
    
    # lists out every email
    email_path = emails_path + f'{email}'

    # open file
    mail = open(email_path,"r")
    # each line is a line of the text file
    lines = mail.readlines()
    mail.close()

    # Extracting Features
    message_id = [line for line in lines if 'Message-ID:' in line]    
    date = [line for line in lines if 'Date:' in line]   
    from_email = [line for line in lines if 'From:' in line]    
    subject = [line for line in lines if 'Subject:' in line]
    encoding = [line for line in lines if 'Content-Transfer-Encoding:' in line]

    # Getting the body of the email
    body_start = [i for i,line in enumerate(lines) if 'X-FileName:' in line][0]
    body = lines[body_start + 2: ]
    body = ' '.join(body)

    # list of words
    body = body.split()

    # count number of words
    body_len = len(body)

    # emails that were as part of the email list
    to_email = [line for line in lines if 'To:' in line]   
    emailed = []
    for word in body:
        if '@' in str(word):
            emailed.append(word.replace(' ', ''))

    if len(to_email) < len(emailed):
        to_email = emailed

    # Collecting email bodies
    kbodies = kbodies.append({'message_id': message_id,
                            'body' : body},
                          ignore_index = True)
    
    # Table listing sender and recipient
    kexchange = kexchange.append({'message_id' : message_id,
                                'from_email' : from_email,
                               'to_email' : to_email},
                              ignore_index = True)
    
    # Contains the email metadata
    kmetadata = kmetadata.append({'message_id' : message_id,
                                'date' : date,
                                'length' : body_len,
                                'subject' : subject,
                                'encoding' : encoding}, 
                               ignore_index = True)    

In [5]:
# Copy the data
kexchange2 = kexchange.copy()
kmetadata2 = kmetadata.copy()

In [46]:
# Output after data scraping
kexchange.head()

,from_email,message_id,to_email,new_to
0,rosalee.fleming@enron.com,<4353541.1075840278810.JavaMail.evans@thyme>,"[To: david.haug@enron.com\n, X-To: David Haug\...",[david.haug@enron.com]
1,rosalee.fleming@enron.com,<7918822.1075840279079.JavaMail.evans@thyme>,"[To: ealvittor@yahoo.com\n, X-To: ealvittor@ya...",[ealvittor@yahoo.com]
2,rosalee.fleming@enron.com,<2796536.1075840281350.JavaMail.evans@thyme>,"[To: mrslinda@lplpi.com\n, X-To: mrslinda@lplp...",[mrslinda@lplpi.com]
3,rosalee.fleming@enron.com,<25179713.1075840281374.JavaMail.evans@thyme>,"[To: philippe.bibi@enron.com, jay.fitzgerald@e...","[philippe.bibi@enron.com, jay.fitzgerald@enron..."
4,rosalee.fleming@enron.com,<6940466.1075840281398.JavaMail.evans@thyme>,[Bannantine/ENRON_DEVELOPMENT@ENRON_DEVELOPMEN...,[Bannantine/ENRON_DEVELOPMENT@ENRON_DEVELOPMEN...


In [26]:
# Data Cleaning
kexchange['from_email'] = kexchange['from_email'].apply(lambda data: data[0].replace('From:','').replace('\n', ''))
kexchange['message_id'] = kexchange['message_id'].apply(lambda data: data[0].replace('Message-ID:','').replace('\n', ''))
kexchange['new_to'] = kexchange['to_email'].apply(lambda ex : ex[0].replace('To:', '').replace('\n', '').replace(' ', '').split(','))

In [29]:
# Reformat the dataframe

new_exchange = pd.DataFrame()
for index, row in kexchange.iterrows():
    for item in row['new_to']:
        if '@' in item:
            new_exchange = new_exchange.append({'from_email' : row.from_email,
                                                'message_id' : row.message_id,
                                                'to_email' : item}, 
                                               ignore_index = True)

In [45]:
# Formatted data
new_exchange.head()

,from_email,message_id,to_email
0,rosalee.fleming@enron.com,<4353541.1075840278810.JavaMail.evans@thyme>,david.haug@enron.com
1,rosalee.fleming@enron.com,<7918822.1075840279079.JavaMail.evans@thyme>,ealvittor@yahoo.com
2,rosalee.fleming@enron.com,<2796536.1075840281350.JavaMail.evans@thyme>,mrslinda@lplpi.com
3,rosalee.fleming@enron.com,<25179713.1075840281374.JavaMail.evans@thyme>,philippe.bibi@enron.com
4,rosalee.fleming@enron.com,<25179713.1075840281374.JavaMail.evans@thyme>,jay.fitzgerald@enron.com


In [31]:
# All unique emails used by Kenneth Lay
new_exchange['from_email'].unique()

array([' rosalee.fleming@enron.com', ' tori.wells@enron.com',
       ' kenneth.lay@enron.com'], dtype=object)

In [ ]:
# Data Cleaning
kmetadata['date'] = kmetadata['date'].apply(lambda data: data[0].replace('Date:','').replace('\n', ''))
kmetadata['encoding'] = kmetadata['encoding'].apply(lambda data: data[0].replace('Content-Transfer-Encoding:','').replace('\n', ''))
kmetadata['message_id'] = kmetadata['message_id'].apply(lambda data: data[0].replace('Message-ID:','').replace('\n', ''))
kmetadata['subject'] = kmetadata['subject'].apply(lambda data: data[0].replace('Subject:','').replace('\n', ''))

In [34]:
# Date, Encoding, Length, Message ID, and Subject of each Email
kmetadata.head()

,date,encoding,length,message_id,subject
0,"Wed, 13 Dec 2000 06:38:00 -0800 (PST)",quoted-printable,376.0,<4353541.1075840278810.JavaMail.evans@thyme>,Re: Succession Plan
1,"Tue, 28 Nov 2000 01:22:00 -0800 (PST)",7bit,65.0,<7918822.1075840279079.JavaMail.evans@thyme>,Rocky Carroll
2,"Tue, 20 Jun 2000 11:00:00 -0700 (PDT)",7bit,90.0,<2796536.1075840281350.JavaMail.evans@thyme>,VIP Reception and Photo Op
3,"Tue, 20 Jun 2000 09:23:00 -0700 (PDT)",7bit,163.0,<25179713.1075840281374.JavaMail.evans@thyme>,Molecular Electronics Corp. Working Lunch
4,"Tue, 20 Jun 2000 06:26:00 -0700 (PDT)",7bit,318.0,<6940466.1075840281398.JavaMail.evans@thyme>,"Re: EXECUTIVE COMMITTEE MEETING - MONDAY, JUN..."


In [ ]:
# Reformatting the Message ID
kbodies['message_id'] = kbodies['message_id'].apply(lambda data: data[0].replace('Message-ID:','').replace('\n', ''))

In [44]:
kbodies.head()

,body,message_id
0,"[We, did, receive, this, from, David., David, ...",<4353541.1075840278810.JavaMail.evans@thyme>
1,"[Hi, Liz, -, I, had, a, very, interesting, con...",<7918822.1075840279079.JavaMail.evans@thyme>
2,"[Linda,, before, we, knew, that, you, and, Ken...",<2796536.1075840281350.JavaMail.evans@thyme>
3,"[On, Tuesday,, July, 25,, I, am, meeting, with...",<25179713.1075840281374.JavaMail.evans@thyme>
4,"[Katherine, Brown, 06/20/2000, 01:07, PM, To:,...",<6940466.1075840281398.JavaMail.evans@thyme>


In [40]:
# Merging teh datasets
kenneth_lay = pd.merge(new_exchange, kmetadata, on='message_id', how='left')
kenneth_lay = pd.merge(kenneth_lay, kbodies, on='message_id', how='left')

In [48]:
# Convert to CSV for the Neo4j graph
kenneth_lay.to_csv('kenneth_lay.csv')

In [49]:
kenneth_lay.head(2)

,from_email,message_id,to_email,date,encoding,length,subject,body
0,rosalee.fleming@enron.com,<4353541.1075840278810.JavaMail.evans@thyme>,david.haug@enron.com,"Wed, 13 Dec 2000 06:38:00 -0800 (PST)",quoted-printable,376.0,Re: Succession Plan,"[We, did, receive, this, from, David., David, ..."
1,rosalee.fleming@enron.com,<7918822.1075840279079.JavaMail.evans@thyme>,ealvittor@yahoo.com,"Tue, 28 Nov 2000 01:22:00 -0800 (PST)",7bit,65.0,Rocky Carroll,"[Hi, Liz, -, I, had, a, very, interesting, con..."
